In [38]:
import pandas as pd
# df = pd.read_csv('90d.csv')
df = pd.read_csv('6M.csv')

df['range'] = (df['high'] - df['low']) * 0.5
df['range_shift1'] = df['range'].shift(1)
df['long_target'] = df['open'] + df['range_shift1']
df['short_target'] = df['open'] - df['range_shift1']
df['close-open'] = df['close'] - df['open']

# 양봉인지 여부.
df['is_green'] = df['close-open'] >= 0



In [39]:
def long_sell_price(x):
    if x['is_green']:
        # 오름봉이면, 최대값에서 100불이 빠졌을때 익절한다.
        return x['high'] - 100 if x['close'] <= x['high'] - 100 else x['close']
    else:
        # 내린봉이면. low 가 진입가보다 -30불이상 더 낮으면 손절한다.
        return x['long_target'] - 30 if x['low'] <= x['long_target'] - 30 else x['close']


def short_sell_price(x):
    if not x['is_green']:
        # 내림봉이면 익절, 최대값에서 100불이 빠졌을때 익절한다.
        return x['low'] + 100 if x['close'] >= x['low'] + 100 else x['close']
    else:
        # 오름봉이면 손절. low 가 진입가보다 -30불이상 더 낮으면 손절한다.
        return x['short_target'] + 30 if x['high'] >= x['short_target'] + 30 else x['close']

In [40]:
import numpy as np
# 롱타겟
# df['high'] > df['long_target'
df['sell_price_long'] = df.apply(lambda x: long_sell_price(x), axis=1)
df['pnl_long'] = np.where(df['high'] > df['long_target'], df['sell_price_long'] - df['long_target'], 0)
df['profit_long%'] = df['pnl_long'] / df['long_target']
df['ror_long'] = np.where(df['high'] > df['long_target'], df['sell_price_long'] / df['long_target'], 1)

# 숏타켓
# df['low'] < df['short_target']
df['sell_price_short'] = df.apply(lambda x: short_sell_price(x), axis=1)
df['pnl_short'] = np.where(df['low'] < df['short_target'], df['short_target'] - df['sell_price_short'], 0)
df['profit_short%'] = df['pnl_short'] / df['short_target']
df['ror_short'] = np.where(df['low'] < df['short_target'], df['short_target'] / df['sell_price_short'], 1)

In [41]:
# 1보다 크면 수익있는것.
df['long_win'] = np.where(df['ror_long'] > 1, 1, 0)
df['long_lose'] = np.where(df['ror_long'] < 1, 1, 0)
df['short_win'] = np.where(df['ror_short'] > 1, 1, 0)
df['short_lose'] = np.where(df['ror_short'] < 1, 1, 0)
df['cum_long_win'] = df['long_win'].cumsum()
df['cum_long_lose'] = df['long_lose'].cumsum()
df['cum_short_win'] = df['short_win'].cumsum()
df['cum_short_lose'] = df['short_lose'].cumsum()
df['cumprod_long'] = df['ror_long'].cumprod()
df['cumprod_short'] = df['ror_short'].cumprod()

# print(df.to_string())
print(df)



     Unnamed: 0   open_time     open    close     high      low      volume  \
0             0  1562025600  10570.5  10836.5  11462.0   9661.5  93432224.0   
1             1  1562112000  10836.5  11989.5  12029.5  10836.5  88350286.0   
2             2  1562198400  11989.5  11161.0  12068.5  11062.0  62613634.0   
3             3  1562284800  11161.0  10986.0  11401.5  10772.5  58569421.0   
4             4  1562371200  10986.0  11236.0  11728.0  10962.5  56837914.0   
..          ...         ...      ...      ...      ...      ...         ...   
174         174  1577059200   7503.5   7314.0   7687.5   7267.5  78395903.0   
175         175  1577145600   7314.0   7249.0   7419.0   7158.0  59308981.0   
176         176  1577232000   7249.0   7194.0   7259.5   7118.0  32256640.0   
177         177  1577318400   7194.0   7191.0   7425.0   7147.0  33206699.0   
178         178  1577404800   7191.0   7170.5   7267.5   7067.0  34611962.0   

           date   range  range_shift1  ...  long_wi